In [1]:
import numpy as np
import math
import re
from collections import Counter
from functools import cmp_to_key

In [2]:
with open("10-input", "r") as file:
    lines = file.readlines()
data_raw = [line.replace("\n", "") for line in lines]
data_raw = "\n".join(data_raw)
data_raw

'F|J-F|.|.F-F-7-7F--|77F.7.77F|-L7-|7|-|-F-L7.7FJF-7-JF7.77-F.7F7.FLF7-FF-L7F--F-J-..J-FJ-FLF7.FF.-F7-|-.F7--77F---7-LF7FFL7.F7|7FJ-FL-.-FJ-F\n-L7-FL7--77J|.7.LJFJFLL.F-F-7JLL7-|-|F|L-.L||||7LJ.LF||LJL|L7FF.F|L||.|7-LFFJ.F7L7FJ|.7.F7.||J-F--|J7LLJJL-|-LL.L-7LLJ7-JFLF---FJ|FJ.L77L|J|\nLLF-7LJ77|.77FJ-JLL-|-LLJF.JJJ.F7L|7L7.FJ.F-7J|77F|--J7JFFJFLLL|7JF7F--J.||L7.L7FJ-7|-F.L-FJ|LLL-FF77FJL-.-JLLL7.F|.-L-7L7|J7|L|7.J-L-JJ.-FJ\n.|F7L-L7J77JLJ.|7|FFL-7|7L7F|JFLJ.LJ|L-LJF|.|J||L|7-L77.F7J7|FLL--||7JF-|-7L-FJJL.L7JFLJ.|L7L-7FF|7|7F7FLF.|.FF.F7L||LLF-LJJ-7||L-7||7|J.|||\nF7JJFF7LFF|-JFF-7FL7|-LLJJJFF7-L|FJ-JJFLF-F-L--J.|.|.L|77J.L7|-L.F|L77JFJ|-J|JLFL-7|7L-|.FL|F7L7F7-L-JL-F7FFF77F-F7-|-L|7-J7.LFJJFJF77FFJLLJ\nFJ.|FLJ-7L-7|||FJ-||F7..||..|77FF7|FJ|L7|J..FFLL---F7-LJ|.|-J|.FF.L7||-J7|JL7|FLF-F|LJ-F-7JLJ|FJ||7|-F-FJ|F-JL77JL7F-7FLFJ|F77FJF|JL7.LJL7.7\nF7-FL|F-7JFL-7FJLFFJFJ.-|F--|JJJLF-|.|.JJ-F7FJL|||FJJ-FFJ.|77L-F-F-JL-7LF777JFF7|---7FF|FJ7-FJ|FJL7F7J-L7|L--7|F-7L7.|7JL--|-L|.||FF|7JL-LF7\n|J.FJ

In [7]:
test_data_raw = """..F7.
.FJ|.
SJ.L7
|F--J
LJ..."""

def preprocess_data (data):
    rows = [list(row) for row in data.split("\n")]
    return np.array(rows)

test_data = preprocess_data(test_data_raw)
test_data

array([['.', '.', 'F', '7', '.'],
       ['.', 'F', 'J', '|', '.'],
       ['S', 'J', '.', 'L', '7'],
       ['|', 'F', '-', '-', 'J'],
       ['L', 'J', '.', '.', '.']], dtype='<U1')

In [9]:
data = preprocess_data(data_raw)
data

array([['F', '|', 'J', ..., 'J', '-', 'F'],
       ['-', 'L', '7', ..., '|', 'J', '|'],
       ['L', 'L', 'F', ..., '-', 'F', 'J'],
       ...,
       ['7', '|', 'F', ..., 'F', '-', '.'],
       ['L', '|', 'J', ..., '|', 'L', '|'],
       ['L', 'L', 'J', ..., '|', '.', 'J']], dtype='<U1')

In [46]:

def solution (data):
    s_loc = np.where(data == "S")
    s_loc = (s_loc[0][0], s_loc[1][0])

    def is_in_field (x,y,data):
       if (x >= 0) and (x < data.shape[0]) and (y >= 0) and (y < data.shape[1]):
          return True
       else:
          return False    
    
    def next_step(x,y, distance, data, record):
        if is_in_field(x,y, data) and record[x,y] == '':
            record[x,y] = distance

            pipe = data[x,y]
            # | is a vertical pipe connecting north and south.
            if pipe == "|":
                next_step(x-1, y, distance+1, data, record)
                next_step(x+1, y, distance+1, data, record)
            # - is a horizontal pipe connecting east and west.
            elif pipe == "-":
                next_step(x, y-1, distance+1, data, record)
                next_step(x, y+1, distance+1, data, record)
            # L is a 90-degree bend connecting north and east.
            elif pipe == "L":
                next_step(x-1, y, distance+1, data, record)
                next_step(x, y+1, distance+1, data, record)
               
            # J is a 90-degree bend connecting north and west.
            elif pipe == "J":
                next_step(x-1, y, distance+1, data, record)
                next_step(x, y-1, distance+1, data, record)
            # 7 is a 90-degree bend connecting south and west.
            elif pipe == "7":
                next_step(x+1, y, distance+1, data, record)
                next_step(x, y-1, distance+1, data, record)
            # F is a 90-degree bend connecting south and east.
            elif pipe == "F":
                next_step(x+1, y, distance+1, data, record)
                next_step(x, y+1, distance+1, data, record)

            elif pipe == 'S':
                if is_in_field(x-1, y, data) and data[x-1, y] in ["|", "7", 'F']:
                    next_step(x-1, y, distance+1, data, record)
                if is_in_field(x+1, y, data) and data[x+1, y] in ["|", "L", 'J']:
                    next_step(x+1, y, distance+1, data, record)
                if is_in_field(x, y-1, data) and data[x, y-1] in ["-", "L", 'F']:
                    next_step(x, y-1, distance+1, data, record)
                if is_in_field(x, y+1, data) and data[x, y+1] in ["-", "7", 'J']:
                    next_step(x, y+1, distance+1, data, record)

    record = np.zeros_like(data,dtype='U64')
    next_step(s_loc[0], s_loc[1], 0, data, record)

    return record


solution = solution(data)
# solution = solution(test_data)
# display(winnings)

display(solution)
solution[solution == ''] = '0'
print(np.ceil(solution.astype(int).max()/2))

RecursionError: maximum recursion depth exceeded in comparison

In [49]:

def solution (data):
    s_loc = np.where(data == "S")
    s_loc = (s_loc[0][0], s_loc[1][0])

    def is_in_field (x,y,data):
       if (x >= 0) and (x < data.shape[0]) and (y >= 0) and (y < data.shape[1]):
          return True
       else:
          return False    
    
    def next_step(x,y, data, record):
        distance = 0
        last = (x,y)
        record[x,y] = distance
        distance += 1
        if is_in_field(x-1, y, data) and data[x-1, y] in ["|", "7", 'F']:
            x = x-1
            y = y
        elif is_in_field(x+1, y, data) and data[x+1, y] in ["|", "L", 'J']:
            x = x+1
            y = y
        elif is_in_field(x, y-1, data) and data[x, y-1] in ["-", "L", 'F']:
            x = x
            y = y-1
        elif is_in_field(x, y+1, data) and data[x, y+1] in ["-", "7", 'J']:
            x = x
            y = y+1

        while is_in_field(x,y, data) and record[x,y] == '':
            record[x,y] = distance
            distance += 1
            pipe = data[x,y]
            # | is a vertical pipe connecting north and south.
            last_tmp = (x,y)
            if pipe == "|":
                x = x-1 if last[0] == x+1 else x+1
                
            # - is a horizontal pipe connecting east and west.
            elif pipe == "-":
                y = y-1 if last[1] == y+1 else y+1
            # L is a 90-degree bend connecting north and east.
            elif pipe == "L":
                if last[0] == x-1:
                    y = y+1
                else:
                    x = x-1
               
            # J is a 90-degree bend connecting north and west.
            elif pipe == "J":
                if last[0] == x-1:
                    y = y-1
                else:
                    x = x-1
            # 7 is a 90-degree bend connecting south and west.
            elif pipe == "7":
                if last[0] == x+1:
                    y = y-1
                else:
                    x = x+1
            # F is a 90-degree bend connecting south and east.
            elif pipe == "F":
                if last[0] == x+1:
                    y = y+1
                else:
                    x = x+1
            last = last_tmp
            

    record = np.zeros_like(data,dtype='U64')
    next_step(s_loc[0], s_loc[1], data, record)

    return record


solution = solution(data)
# solution = solution(test_data)
# display(winnings)

display(solution)
solution[solution == ''] = '0'
print(np.ceil(solution.astype(int).max()/2))

array([['', '', '', ..., '', '', ''],
       ['', '', '', ..., '', '', ''],
       ['', '', '', ..., '', '', ''],
       ...,
       ['', '', '', ..., '', '', ''],
       ['', '', '', ..., '', '', ''],
       ['', '', '', ..., '', '', '']], dtype='<U64')

6907.0


In [50]:
test_data_raw_2 ="""FF7FSF7F7F7F7F7F---7
L|LJ||||||||||||F--J
FL-7LJLJ||||||LJL-77
F--JF--7||LJLJ7F7FJ-
L---JF-JLJ.||-FJLJJ7
|F|F-JF---7F7-L7L|7|
|FFJF7L7F-JF7|JL---7
7-L-JL7||F7|L7F-7F7|
L.L7LFJ|||||FJL7||LJ
L7JLJL-JLJLJL--JLJ.L"""

test_data_2 = preprocess_data(test_data_raw_2)
test_data_2

array([['F', 'F', '7', 'F', 'S', 'F', '7', 'F', '7', 'F', '7', 'F', '7',
        'F', '7', 'F', '-', '-', '-', '7'],
       ['L', '|', 'L', 'J', '|', '|', '|', '|', '|', '|', '|', '|', '|',
        '|', '|', '|', 'F', '-', '-', 'J'],
       ['F', 'L', '-', '7', 'L', 'J', 'L', 'J', '|', '|', '|', '|', '|',
        '|', 'L', 'J', 'L', '-', '7', '7'],
       ['F', '-', '-', 'J', 'F', '-', '-', '7', '|', '|', 'L', 'J', 'L',
        'J', '7', 'F', '7', 'F', 'J', '-'],
       ['L', '-', '-', '-', 'J', 'F', '-', 'J', 'L', 'J', '.', '|', '|',
        '-', 'F', 'J', 'L', 'J', 'J', '7'],
       ['|', 'F', '|', 'F', '-', 'J', 'F', '-', '-', '-', '7', 'F', '7',
        '-', 'L', '7', 'L', '|', '7', '|'],
       ['|', 'F', 'F', 'J', 'F', '7', 'L', '7', 'F', '-', 'J', 'F', '7',
        '|', 'J', 'L', '-', '-', '-', '7'],
       ['7', '-', 'L', '-', 'J', 'L', '7', '|', '|', 'F', '7', '|', 'L',
        '7', 'F', '-', '7', 'F', '7', '|'],
       ['L', '.', 'L', '7', 'L', 'F', 'J', '|', '|', '|', '|', '

In [88]:
def solution (data):
    s_loc = np.where(data == "S")
    s_loc = (s_loc[0][0], s_loc[1][0])

    def is_in_field (x,y,data):
       if (x >= 0) and (x < data.shape[0]) and (y >= 0) and (y < data.shape[1]):
          return True
       else:
          return False    
    
    def get_entry_direction(source, target):
        if target[0] > source[0]:
            return "N"
        elif target[0] < source[0]:
            return "S"
        elif target[1] > source[1]:
            return "W"
        elif target[1] < source[1]:
            return "E"

    def next_step(x,y, data, record):
        distance = 0
        last = (x,y)
        record[x,y] = distance
        loop = [(x,y)]
        entries = [None]
        distance += 1
        if is_in_field(x-1, y, data) and data[x-1, y] in ["|", "7", 'F']:
            x = x-1
            y = y
        elif is_in_field(x+1, y, data) and data[x+1, y] in ["|", "L", 'J']:
            x = x+1
            y = y
        elif is_in_field(x, y-1, data) and data[x, y-1] in ["-", "L", 'F']:
            x = x
            y = y-1
        elif is_in_field(x, y+1, data) and data[x, y+1] in ["-", "7", 'J']:
            x = x
            y = y+1

        

        while is_in_field(x,y, data) and record[x,y] == '':
            record[x,y] = distance
            before = loop[-1]
            loop.append((x,y))
            entries.append(get_entry_direction(before, (x,y)))
            distance += 1
            pipe = data[x,y]
            # | is a vertical pipe connecting north and south.
            last_tmp = (x,y)
            if pipe == "|":
                x = x-1 if last[0] == x+1 else x+1
                
            # - is a horizontal pipe connecting east and west.
            elif pipe == "-":
                y = y-1 if last[1] == y+1 else y+1
            # L is a 90-degree bend connecting north and east.
            elif pipe == "L":
                if last[0] == x-1:
                    y = y+1
                else:
                    x = x-1
               
            # J is a 90-degree bend connecting north and west.
            elif pipe == "J":
                if last[0] == x-1:
                    y = y-1
                else:
                    x = x-1
            # 7 is a 90-degree bend connecting south and west.
            elif pipe == "7":
                if last[0] == x+1:
                    y = y-1
                else:
                    x = x+1
            # F is a 90-degree bend connecting south and east.
            elif pipe == "F":
                if last[0] == x+1:
                    y = y+1
                else:
                    x = x+1
            last = last_tmp

        entries[0] = get_entry_direction(loop[-1], loop[0])
        return loop, entries
            
    def get_start_pipe (s, first, last):
        if first[0] == last[0]:
            return "-"
        elif first[1] == last[1]:
            return "|"
        # L is a 90-degree bend connecting north and east.
        elif ((first == (s[0] - 1 , s[1]) ) and (last == (s[0], s[1]+1)) ) or ((last == (s[0] - 1 , s[1])) and (first == (s[0], s[1]+1) )):
            return "L"        
        # J is a 90-degree bend connecting north and west.
        elif ((first == (s[0] , s[1]-1) ) and (last == (s[0]-1, s[1])) ) or ((last == (s[0], s[1]-1)) and (first == (s[0]-1, s[1]) )):
            return "J"
        # 7 is a 90-degree bend connecting south and west.
        elif ((first == (s[0] , s[1]-1) ) and (last == (s[0]+1, s[1])) ) or ((last == (s[0], s[1]-1)) and (first == (s[0]+1, s[1]) )):
            return "7"
        # F is a 90-degree bend connecting south and east.
        elif ((first == (s[0] + 1 , s[1]) ) and (last == (s[0], s[1]+1)) ) or ((last == (s[0] + 1 , s[1])) and (first == (s[0], s[1]+1) )):
            return "F"

    def get_enclosing_tags (symbol, pos, entry):
        if symbol == "|":
            if entry == 'N':
                return [(pos[0], pos[1]-1)], [(pos[0], pos[1]+1)]
            elif entry == 'S':
                return [(pos[0], pos[1]+1)], [(pos[0], pos[1]-1)]
                    
        # - is a horizontal pipe connecting east and west.
        elif symbol == "-":
            if entry == 'W':
                return [(pos[0]+1, pos[1])], [(pos[0]-1, pos[1])]
            elif entry == 'E':
                return [(pos[0]-1, pos[1])], [(pos[0]+1, pos[1]-1)]
        # L is a 90-degree bend connecting north and east.
        elif symbol == "L":
            if entry == 'E':
                return [], [(pos[0]+1, pos[1]), (pos[0]+1, pos[1]-1), (pos[0], pos[1]-1)]
            elif entry == 'N':
                return [(pos[0]+1, pos[1]), (pos[0]+1, pos[1]-1), (pos[0], pos[1]-1)], []
            
        # J is a 90-degree bend connecting north and west.
        elif symbol == "J":
            if entry == 'N':
                return [], [(pos[0]+1, pos[1]), (pos[0]+1, pos[1]+1), (pos[0], pos[1]+1)]
            elif entry == 'W':
                return [(pos[0]+1, pos[1]), (pos[0]+1, pos[1]+1), (pos[0], pos[1]+1)], []
        # 7 is a 90-degree bend connecting south and west.
        elif symbol == "7":
            if entry == 'W':
                return [], [(pos[0]-1, pos[1]), (pos[0]-1, pos[1]+1), (pos[0], pos[1]+1)]
            elif entry == 'S':
                return [(pos[0]-1, pos[1]), (pos[0]-1, pos[1]+1), (pos[0], pos[1]+1)], []
        # F is a 90-degree bend connecting south and east.
        elif symbol == "F":
            if entry == 'S':
                return [], [(pos[0]-1, pos[1]), (pos[0]-1, pos[1]-1), (pos[0], pos[1]-1)]
            elif entry == 'E':
                return [(pos[0]-1, pos[1]), (pos[0]-1, pos[1]-1), (pos[0], pos[1]-1)], []

    record = np.zeros_like(data,dtype='U64')
    loop, entries = next_step(s_loc[0], s_loc[1], data, record)
    start_pipe = get_start_pipe(s_loc, loop[1], loop[-1])

    # mark I and O
    for idx, (x,y) in enumerate(loop):
        pipe = data[x,y]
        if pipe == 'S':
            pipe = start_pipe
        Is, Os = get_enclosing_tags(pipe, (x,y), entries[idx])
        for a,b in Is:
            if is_in_field(a, b, data) and record[a,b] == '':
                record[a,b] = 'I'
        for a,b in Os:
            if is_in_field(a, b, data) and record[a,b] == '':
                record[a,b] = 'O'       
    
    # fill uncompleted areas
    x_coords, y_coords = np.where(record == '')
    blanks = list(zip(x_coords, y_coords ))

    while blanks:
        for blank in blanks:
            for neighbor in [(blank[0]+1, blank[1]), (blank[0]-1, blank[1]), (blank[0], blank[1]+1), (blank[0], blank[1]-1)]:
                if is_in_field(neighbor[0], neighbor[1] , data) and (record[neighbor] in ["I", "O"]):
                    record[blank] = record[neighbor]
        x_coords, y_coords = np.where(record == '')
        blanks = list(zip(x_coords, y_coords ))


    # print(np.unique(record))
    # print(start_pipe)
    # print(loop)
    # print(entries)
    return record


sol = solution(data)
# sol = solution(test_data)
# sol = solution(test_data_2)
# display(solution)
print("Number Os", np.where(sol == 'O')[0].shape[0])
print("Number Is", np.where(sol == 'I')[0].shape[0])
# Take smaller number because of empty space 
display(sol)


Number Os 5245
Number Is 541


array([['O', 'O', 'O', ..., 'O', 'O', 'O'],
       ['O', 'O', 'O', ..., 'O', 'O', 'O'],
       ['O', 'O', 'O', ..., 'O', 'O', 'O'],
       ...,
       ['O', 'O', 'O', ..., 'O', 'O', 'O'],
       ['O', 'O', 'O', ..., 'O', 'O', 'O'],
       ['O', 'O', 'O', ..., 'O', 'O', 'O']], dtype='<U64')

In [59]:
loop = [(2, 0), (3, 0), (4, 0), (4, 1), (3, 1), (3, 2), (3, 3), (3, 4), (2, 4), (2, 3), (1, 3), (0, 3), (0, 2), (1, 2), (1, 1), (2, 1)]
def get_start_pipe (s, first, last):
    if first[0] == last[0]:
        return "-"
    elif first[1] == last[1]:
        return "|"
    # L is a 90-degree bend connecting north and east.
    elif ((first == (s[0] - 1 , s[1]) ) and (last == (s[0], s[1]+1)) ) or ((last == (s[0] - 1 , s[1])) and (first == (s[0], s[1]+1) )):
        return "L"        
    # J is a 90-degree bend connecting north and west.
    elif ((first == (s[0] , s[1]-1) ) and (last == (s[0]-1, s[1])) ) or ((last == (s[0], s[1]-1)) and (first == (s[0]-1, s[1]) )):
        return "J"
    # 7 is a 90-degree bend connecting south and west.
    elif ((first == (s[0] , s[1]-1) ) and (last == (s[0]+1, s[1])) ) or ((last == (s[0], s[1]-1)) and (first == (s[0]+1, s[1]) )):
        return "7"
    # F is a 90-degree bend connecting south and east.
    elif ((first == (s[0] + 1 , s[1]) ) and (last == (s[0], s[1]+1)) ) or ((last == (s[0] + 1 , s[1])) and (first == (s[0], s[1]+1) )):
        return "F"  
    
get_start_pipe(loop[0], loop[1], loop[-1])






'F'

In [64]:
def get_enclosing_tags (symbol, pos, entry):
    if symbol == "|":
        if entry == 'N':
            return [(pos[0], pos[1]-1)], [(pos[0], pos[1]+1)]
        elif entry == 'S':
            return [(pos[0], pos[1]+1)], [(pos[0], pos[1]-1)]
                
    # - is a horizontal pipe connecting east and west.
    elif symbol == "-":
        if entry == 'W':
            return [(pos[0]+1, pos[1])], [(pos[0]-1, pos[1])]
        elif entry == 'E':
            return [(pos[0]-1, pos[1])], [(pos[0]+1, pos[1]-1)]
    # L is a 90-degree bend connecting north and east.
    elif symbol == "L":
        if entry == 'E':
            return [], [(pos[0]+1, pos[1]), (pos[0]+1, pos[1]-1), (pos[0], pos[1]-1)]
        elif entry == 'N':
            return [(pos[0]+1, pos[1]), (pos[0]+1, pos[1]-1), (pos[0], pos[1]-1)], []
        
    # J is a 90-degree bend connecting north and west.
    elif symbol == "J":
        if entry == 'N':
            return [], [(pos[0]+1, pos[1]), (pos[0]+1, pos[1]+1), (pos[0], pos[1]+1)]
        elif entry == 'W':
            return [(pos[0]+1, pos[1]), (pos[0]+1, pos[1]+1), (pos[0], pos[1]+1)], []
    # 7 is a 90-degree bend connecting south and west.
    elif symbol == "7":
        if entry == 'W':
            return [], [(pos[0]-1, pos[1]), (pos[0]-1, pos[1]+1), (pos[0], pos[1]+1)]
        elif entry == 'S':
            return [(pos[0]-1, pos[1]), (pos[0]-1, pos[1]+1), (pos[0], pos[1]+1)], []
    # F is a 90-degree bend connecting south and east.
    elif symbol == "F":
        if entry == 'S':
            return [], [(pos[0]-1, pos[1]), (pos[0]-1, pos[1]-1), (pos[0], pos[1]-1)]
        elif entry == 'E':
            return [(pos[0]-1, pos[1]), (pos[0]-1, pos[1]-1), (pos[0], pos[1]-1)], []
        
get_enclosing_tags("F", (2,0), "S")

([], [(1, 0), (1, -1), (2, -1)])

In [87]:
test_case1 = """...........
.S-------7.
.|F-----7|.
.||.....||.
.||.....||.
.|L-7.F-J|.
.|..|.|..|.
.L--J.L--J.
..........."""
test_case1 = preprocess_data(test_case1)
test_case1_solution = solution(test_case1)
print("Number Os", np.where(test_case1_solution == 'O')[0].shape[0])
print("Number Is", np.where(test_case1_solution == 'I')[0].shape[0])

test_case2 = """..........
.S------7.
.|F----7|.
.||....||.
.||....||.
.|L-7F-J|.
.|..||..|.
.L--JL--J.
.........."""
test_case2 = preprocess_data(test_case2)
test_case2_solution = solution(test_case2)
print("Number Os", np.where(test_case2_solution == 'O')[0].shape[0])
print("Number Is", np.where(test_case2_solution == 'I')[0].shape[0])

test_case3 = """.F----7F7F7F7F-7....
.|F--7||||||||FJ....
.||.FJ||||||||L7....
FJL7L7LJLJ||LJ.L-7..
L--J.L7...LJS7F-7L7.
....F-J..F7FJ|L7L7L7
....L7.F7||L7|.L7L7|
.....|FJLJ|FJ|F7|.LJ
....FJL-7.||.||||...
....L---J.LJ.LJLJ..."""
test_case3 = preprocess_data(test_case3)
test_case3_solution = solution(test_case3)
print("Number Os", np.where(test_case3_solution == 'O')[0].shape[0])
print("Number Is", np.where(test_case3_solution == 'I')[0].shape[0])

test_case4 = """FF7FSF7F7F7F7F7F---7
L|LJ||||||||||||F--J
FL-7LJLJ||||||LJL-77
F--JF--7||LJLJ7F7FJ-
L---JF-JLJ.||-FJLJJ7
|F|F-JF---7F7-L7L|7|
|FFJF7L7F-JF7|JL---7
7-L-JL7||F7|L7F-7F7|
L.L7LFJ|||||FJL7||LJ
L7JLJL-JLJLJL--JLJ.L"""
test_case4 = preprocess_data(test_case4)
test_case4_solution = solution(test_case4)
print("Number Os", np.where(test_case4_solution == 'O')[0].shape[0])
print("Number Is", np.where(test_case4_solution == 'I')[0].shape[0])


Number Os 4
Number Is 49
Number Os 4
Number Is 42
Number Os 52
Number Is 8
Number Os 30
Number Is 10
